In [ ]:
import requests
import re
import urllib 
from bs4 import BeautifulSoup
import string
import pandas as pd
import numpy as np

In [ ]:
temporary_links = []
primary_links = []
product_links = []


In [ ]:
url = 'https://www.jtcrussia.ru'
r = requests.get(url)
html_content = r.text
soup = BeautifulSoup(html_content,'html.parser')

In [ ]:
for link in soup.findAll('a', attrs={'href': re.compile("/catalog/")}):
    temporary_links.append(link.get('href'))
temporary_links=temporary_links[1:]

In [ ]:
kio=0
lio=len(temporary_links)
for temporary_link in temporary_links:
    kio+=1
    url='https://www.jtcrussia.ru'+temporary_link
    r = requests.get(url)
    html_content = r.text
    soup = BeautifulSoup(html_content,'html.parser')
    if soup.findAll('li', class_="last")==[]:
        for link in soup.findAll('a', attrs={'href': re.compile("/tools/")}):
            product_links.append(link.get('href'))
    else:       
        a=str(soup.findAll('li', class_="last")[0])
        last_pages=int(a[(a.find('?page='))+6:-11])
        for i in range(1,last_pages+1):
            print(kio,lio,i,last_pages)
            url_vn=url+'?page='+str(i)
            r = requests.get(url_vn)
            html_content = r.text
            soup = BeautifulSoup(html_content,'html.parser')
            for link in soup.findAll('a', attrs={'href': re.compile("/tools/")}):
                product_links.append(link.get('href'))


In [ ]:
product_links_unique=np.unique(product_links)

In [ ]:
titles = []
prices = []
marks=[]
articles=[]
valutes=[]
categories=[]
urles=[]

In [ ]:
mio=0
for product_link in product_links_unique[3323:]:
    mio+=1
    print(mio)
    url = 'https://www.jtcrussia.ru' + product_link
    try:
        r = requests.get(url)
    except requests.exceptions.ConnectionError:
        r.status_code = "Connection refused"
    html_content = r.text
    soup = BeautifulSoup(html_content,'html.parser')

    title = soup.findAll('h1')[-1].text
    titles.append(title)

        
    article=soup.findAll('p',class_='article')[0].text
    articles.append(article)

    price=''
    aaaa=soup.findAll('u')[0].text.split()[:-1]
    for i in range(int(len(aaaa))):
        price=price+aaaa[i]
    prices.append(price)
        
    if len(soup.findAll('h1'))==1:
        lo='iii'
    else:
        lo=str(soup.findAll('h1')[-2]).split()
    
    k=0
    for i in range(int(len(lo))):
        if lo[i]=='<span></span></div>':
            k=i+1
        brand=lo[k:-1]
    b=''
    for i in range(int(len(brand))):
        b=str(b)+' '+brand[i]
    category=b[1:]
    categories.append(category)
        
    urles.append(url)

#product_linkks =[ 'https://www.jtcrussia.ru%s'%x for x in product_links_unique ]

table = pd.DataFrame(list(zip(titles, articles,prices, categories,urles)), columns =['Наименование товара', 'Артикул','Цена','Категория','Ссылка на товар']) 

table.to_excel('jtc 100321.xlsx')